In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional, Tuple, List
from datetime import datetime
import scikitplot as skplt
from itertools import combinations


NOW = datetime.now().strftime('%d_%m_%Y_%H_%M_%S')

In [2]:
def save_plot(plot, name: str):
    try:
        fig = plot.get_figure()
        fig.savefig(f"{name}.png")  
    except AttributeError:
        if type(plot) == list:
            plot.figure.savefig(f"{name}.png")
        else:
            plot.savefig(f"{name}.png")
    

In [3]:
class ExplorationPlot:

    def __init__(self, data):
        self.data = data
    
    def _distribution_plot_1d(self, column: str, save_plot_bool: bool, name: str):
        
        plt.figure()
        plot = sns.histplot(data=self.data, x=column, legend = True, bins = 50)
        if save_plot_bool == True:
            
            save_plot(plot, name)
        

    def _distribution_plot_2d(self, x: str, y: str, save_plot_bool: bool, name: str):
        
        plt.figure()
        plot =  sns.histplot(data=self.data, x=x, y = y, legend = True)

        if save_plot_bool == True:
            save_plot(plot, name)
        

    def _scatter_plot(self, x: str, y: str, save_plot_bool: bool, name: str):
        
        plt.figure()
        plot = sns.scatterplot(data=self.data, x=x, y=y)

        if save_plot_bool == True:
            save_plot(plot, name)


    def _correlation_plot(self, save_plot_bool: bool, name: str):
        
        corr = self.data.corr()
        mask = np.triu(np.ones_like(corr, dtype=bool))
        f, ax = plt.subplots(figsize=(11, 9))
        cmap = sns.diverging_palette(230, 20, as_cmap=True)

        plt.figure()
        plot = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                    square=True, linewidths=.5, cbar_kws={"shrink": .5})

        if save_plot_bool == True:
            save_plot(plot, name)
     
    def _overview_plot(self, save_plot_bool: bool, name: str):
        
        plot = plt.figure()
        plot = sns.PairGrid(self.data)
        
        plot.map_diag(sns.histplot)
        plot.map_offdiag(sns.scatterplot)
        
        if save_plot_bool == True:
            save_plot(plot, name)
    
    def _get_2d_hist_plot(self, plot_2d_names: str, save_plot_2d: bool):
        if plot_2d_names[0] == "None":
            for col_tuple in combinations(list(self.data.columns.values), 2):
                print(col_tuple)
                if save_plot_2d == True:
                    
                    self._distribution_plot_2d(
                                                x = col_tuple[0],
                                                y = col_tuple[1],
                                                save_plot_bool = True,
                                                name = f"./img/hist2d/hist2d_x_{col_tuple[0]}_y_{col_tuple[1]}_{NOW}"                                
                                            )
                
                    #Vyhledavani na obe strany
                    self._distribution_plot_2d(
                                                x = col_tuple[1],
                                                y = col_tuple[0],
                                                save_plot_bool = True,
                                                name = f"./img/hist2d/hist2d_x_{col_tuple[0]}_y_{col_tuple[1]}_{NOW}"                                
                                            )
                else:
                        self._distribution_plot_2d(
                                                    x = col_tuple[0],
                                                    y = col_tuple[1],
                                                    save_plot_bool = False,
                                                    name = f"./img/hist2d/hist2d_x_{col_tuple[0]}_y_{col_tuple[1]}_{NOW}"                                
                                                )
        else:
            for col_tuple in combinations(plot_2d_names, 2):
                if save_plot_2d == True:
                    
                    self._distribution_plot_2d(
                                                x = col_tuple[0],
                                                y = col_tuple[1],
                                                save_plot_bool = True,
                                                name = f"./img/hist2d/hist2d_x_{col_tuple[0]}_y_{col_tuple[1]}_{NOW}"                                
                                            )
                
                    #Vyhledavani na obe strany
                    self._distribution_plot_2d(
                                                x = col_tuple[1],
                                                y = col_tuple[0],
                                                save_plot_bool = True,
                                                name = f"./img/hist2d/hist2d_x_{col_tuple[0]}_y_{col_tuple[1]}_{NOW}"                                
                                            )
                else:
                        self._distribution_plot_2d(
                                                    x = col_tuple[0],
                                                    y = col_tuple[1],
                                                    save_plot_bool = False,
                                                    name = f"./img/hist2d/hist2d_x_{col_tuple[0]}_y_{col_tuple[1]}_{NOW}"                                
                                                )

    def _get_1d_hist_plot(self, save_plot_1d: bool):
        
        
        for col in self.data.columns:
            if save_plot_1d == True:
                self._distribution_plot_1d(
                                            column = col, 
                                            save_plot_bool = True, 
                                            name = f"./img/hist1d/hist1d_{col}_{NOW}"
                                        )
            else:
                self._distribution_plot_1d(
                                            column = col, 
                                            save_plot_bool = False, 
                                            name = f"./img/hist1d/hist1d_{col}_{NOW}"
                                        )
    
    def _create_exploration_report(
                                    self,
                                    plot_2d_names: Optional[List[str]] = ["None"],
                                    save_plot_1d: Optional[bool] = True,
                                    save_plot_2d: Optional[bool] = True,
                                    save_plot_corr: Optional[bool] = True,
                                    save_plot_overview: Optional[bool] = True,  
                            ):

        ## CONSTANTS
        CORR_FILE_NAME = "./img/corr/corr_{NOW}"
        OVERVIEW_FILE_NAME = f"./img/overview/overview_{NOW}"
        
        #self._get_1d_hist_plot(save_plot_1d=save_plot_1d)
        self._get_2d_hist_plot(plot_2d_names=plot_2d_names, save_plot_2d=save_plot_2d)
        
        if save_plot_corr == True:
            self._correlation_plot(save_plot_bool = True, name = CORR_FILE_NAME)
        else:
            self._correlation_plot(save_plot_bool = False, name = CORR_FILE_NAME)
        
        if save_plot_overview == True:
            self._overview_plot(save_plot_bool = True, name = OVERVIEW_FILE_NAME)
        else:
            self._overview_plot(save_plot_bool = False, name = OVERVIEW_FILE_NAME)
        

In [ ]:
class ImpPlot:
  def __init__(self, SMALL_SIZE = None, BIG_SIZE = None):
    self.SMALL_SIZE = SMALL_SIZE
    self.BIG_SIZE = BIG_SIZE
  
  @staticmethod
  def _config_text(SMALL_SIZE: Optional[int] = None, BIG_SIZE: Optional[int] = None):  
    plt.rc('font', size=SMALL_SIZE)
    plt.rc('axes', titlesize=SMALL_SIZE)
    plt.rc('axes', labelsize=SMALL_SIZE)
    plt.rc('legend', fontsize=SMALL_SIZE)
    plt.rc('figure', titlesize=BIG_SIZE)
    
  @staticmethod
  def filter_error_iteration(err, iter, iter_to_plot: int):
    
    return [x for x, y in list(zip(err, iter)) if y == iter_to_plot]
        
  def err_plot(self,   
                  err: np.array,
                  iter: np.array,
                  y_color: str, 
                  y_label: str,
                  iter_to_plot: int,
                  units: Optional[str],
                  y_lim: Optional[list] = None,
                  distribution: Optional[str] = None,
                  tol: Optional[str] = None,
                  name: Optional[str] = None
              ):
    
    y = self.filter_error_iteration(err, iter, iter_to_plot)
    if units is None:
      units = '[-]'
      
    ## INIT FIGURE
    plt.figure()
    
    #Figure grid setup
    fig, ax = plt.subplots(figsize = (20, 10))
    
    if (self.SMALL_SIZE is not None) and (self.SMALL_SIZE is not None):
      self._config_text(SMALL_SIZE = self.SMALL_SIZE, BIG_SIZE = self.BIG_SIZE)
    
    #grid max-min
    plt.xlim([0, len(y) + 1])
    
    if y_lim is not None:
      plt.ylim(y_lim)
    
    #Plot baselines
    plt.axhline(y = 0, color = 'k', linestyle = '-')
    plt.axhline(y = 0.01, color = 'r', linestyle = ':')
    plt.axhline(y = -0.01, color = 'r', linestyle = ':')
    
    #plot legend
    ax.set_xlabel(f"{iter_to_plot}-tá iterace {units}")
    ax.set_ylabel(f"Velikost relativního rozdílu i-té a předchozí iterace")
    ax.set_title(f'Velikost relativního rozdílu i-té a předchozí iterace pro {iter_to_plot}-tý chybějící bod')
    
    #plotting
    plt.plot(y, y_color, label = y_label)
    plt.legend()
    
    save_plot(plt, name = f"img/imp/imp_err_{y_label}_{NOW}")
    
    
    return plt.show()

  
  def imp_plot( self,
                y: np.array,
                y2: np.array,
                y_color: str, 
                y_label: str,
                y2_color: str, 
                y2_label: str,
                units: Optional[str] = None,
                legend_position: Optional[str] = None,
                distribution: Optional[str] = None,
                tol: Optional[str] = None,
                name: Optional[str] = None,
                type_name: Optional[str] = None
              ):
    
    x = np.arange(1, len(y) + 1)
    if units is None:
      units = '[-]'
    
    ## INIT FIGURE
    plt.figure()
    
    #Figure grid setup
    fig, ax = plt.subplots(figsize = (20, 10))
    
    #grid max-min
    lower_lim = min(np.minimum(y, y2)) - 0.5
    upper_lim = max(np.maximum(y, y2)) + 0.5
    
    plt.xlim([0, len(y) + 1])
    plt.ylim([lower_lim, upper_lim])
    
    #Plot baselines
    plt.axhline(y = 0, color = 'k', linestyle = '')
    
    #Plot legend
    ax.set_xlabel("I-tý nezmáný bod [-]")
    ax.set_ylabel(f"Hodnota bodu {units}")
    ax.set_title('Rozdíl skutečné a odhadnuté hodnoty')
    
    #Plot lines
    plt.plot(x, y, y_color, label = y_label)
    plt.plot(x, y2, y2_color, label = y2_label)
    
    plt.legend()
    
    save_plot(plt, name = f"img/imp/imp_{name}_{distribution}_{NOW}")
    
    
    return plt.show()


In [ ]:
class PlotClassificationMetrics:
    def __init__(self, act, pred):
        self.act = act
        self.pred = pred

    @staticmethod
    def _create_string():
        for i in [';', ":", ",", ".", " ", "-"]:
            string = f"img/score/confussion_matrix_{datetime.now().strftime('%d_%m_%Y_%H_%M_%S')}"

        return string

    def _plot_conf_matrix(self):
        
        plot = skplt.metrics.plot_confusion_matrix(self.act, self.pred, normalize=True)
        
        save_plot(plot, self._create_string())

